In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import tkinter as tk
from tkinter import messagebox

In [2]:
dataraw = pd.read_csv("datas_with_clusters.csv")

In [3]:
def get_knn_recommendations(dataraw, indices, distances):
    recommendations = []
    for i in range(1, len(indices[0])):
        recommendations.append({
            "rank": i,
            "category": dataraw.iloc[indices[0][i]]['cluster'],
            "name": dataraw.iloc[indices[0][i]]['name'],
            "artist": dataraw.iloc[indices[0][i]]['artists'],
            "distance": distances[0][i]
        })
    return recommendations

def print_recommendations(recommendations):
    for rec in recommendations:
        print(f"{rec['rank']}. En Yakın Komşu: Kategori: {rec['category']}, {rec['name']}, Mesafe: {rec['distance']}")

In [4]:
numerical_cols = ['acousticness', 'danceability', 'energy', 'instrumentalness',
                 'speechiness', 'tempo', 'valence', 'popularity']
#numerical_cols = ['energy', 'valence' , 'instrumentalness', 'acousticness', 'danceability', 'popularity']

In [5]:
data = dataraw[numerical_cols]
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

In [6]:
knn = NearestNeighbors(n_neighbors=7)  # 5 en yakın komşu
knn.fit(data_scaled)

NearestNeighbors(n_neighbors=7)

In [7]:
cluster_1_data = dataraw[dataraw['cluster'] == 9]
artists_list = cluster_1_data['artists'].apply(ast.literal_eval).explode()

artist_counts = artists_list.value_counts()

print("Cluster 9 artistler:")
print(artist_counts.head())

Cluster 9 artistler:
Metallica                140
AC/DC                     95
Red Hot Chili Peppers     95
Queen                     95
Led Zeppelin              84
Name: artists, dtype: int64


In [8]:
import tkinter as tk
from tkinter import messagebox

def get_recommendations():
    selected_song = entry.get()
    if not selected_song:
        messagebox.showwarning("Uyarı", "Lütfen bir şarkı adı yazın!")
        return

    song_index = dataraw[dataraw['name'] == selected_song].index[0]
    song_features = data_scaled[song_index].reshape(1, -1)
    distances, indices = knn.kneighbors(song_features)
    
    recommendations = get_knn_recommendations(dataraw, indices, distances)
    result_text.delete(1.0, tk.END)
    for rec in recommendations:
        result_text.insert(tk.END, f"{rec['rank']}. Sanatçı: {rec['artist']},Şarkı: {rec['name']}, Doğruluk: {1-rec['distance']}\n")

root = tk.Tk()
root.title("Şarkı Öneri Sistemi")

frame = tk.Frame(root)
frame.pack(pady=20)

label = tk.Label(frame, text="Şarkı Adı Yazın:")
label.pack(side=tk.LEFT)

entry = tk.Entry(frame, width=50)
entry.pack(side=tk.LEFT, padx=10)

recommend_button = tk.Button(root, text="Öneri Getir", command=get_recommendations)
recommend_button.pack(pady=10)

result_text = tk.Text(root, height=10, width=80)
result_text.pack(pady=10)

root.mainloop()